# Export 2019 Recipients and match Canton

In [1]:
import pandas as pd
import mysql.connector

In [2]:
year = 2019

## Connect

In [3]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database = "pharmagelder"
)

## Load Data

In [4]:
df_recipient = pd.read_sql("""SELECT
  rec_id as id,
  rec_name as name,
  rec_address as address,
  rec_location as location,
  rec_type as type,
  trc_name as category,
  sum(tra_value) as value
 FROM transaction
  LEFT JOIN transaction_category ON tra_fk_transaction_category = transaction_category.trc_id
  LEFT JOIN recipient ON tra_fk_recipient = recipient.rec_id
  WHERE tra_year = %s
  GROUP BY tra_fk_recipient, tra_fk_transaction_category
  ORDER BY rec_name DESC""" % year, con=db_connection)

In [5]:
len(df_recipient)

10160

In [6]:
df_verzeichnis = pd.read_csv('Ortschaftenverz.-Rép. Localités.csv')
df_GDENAMK = df_verzeichnis[['KTKZ', 'GDENAMK']].copy().rename(columns={'GDENAMK': 'ORTNAME'})
df_PLZNAMK = df_verzeichnis[['KTKZ', 'PLZNAMK']].copy().rename(columns={'PLZNAMK': 'ORTNAME'})
df_verzeichnis = pd.concat([df_verzeichnis, df_GDENAMK, df_PLZNAMK], sort=False).dropna(axis=0, subset=['ORTNAME'])
df_verzeichnis['ORTNAME'] = df_verzeichnis['ORTNAME'].str.lower()
df_verzeichnis = df_verzeichnis[['KTKZ', 'ORTNAME']].drop_duplicates(subset='ORTNAME')
len(df_verzeichnis)

6665

## Match

In [7]:
#Pivot
df_pivot = pd.pivot_table(df_recipient, values='value', columns='category', index=['id', 'name', 'address', 'location', 'type'])
df_pivot['total'] = df_pivot.sum(axis=1)
df_pivot = df_pivot.reset_index()
df_pivot['location_s'] = df_pivot['location'].str.lower()

In [8]:
df_new = df_pivot.merge(df_verzeichnis, how='left', left_on='location_s', right_on='ORTNAME')
len(df_new)

5916

In [9]:
len(df_new[df_new.KTKZ.isna()])

96

In [10]:
#df_new[df_new.KTKZ.isna()].groupby('location').count().sort_values('name', ascending=False)

## Export

In [11]:
#Write all
df_pivot = df_new.sort_values(['total'], ascending=False)
df_pivot = df_pivot.drop(columns=['location_s', 'ORTNAME']).rename(columns={'KTKZ': 'Canton'})
writer = pd.ExcelWriter('../../data/99. analyzes/excel/recipients/%s_recipients.xlsx' % year, options={'encoding':'utf-8'})
df_pivot[df_pivot.type == 'hcp'].to_excel(writer, 'hcp', index=True)
df_pivot[df_pivot.type == 'hco'].to_excel(writer, 'hco', index=True)
writer.save()

print("finito")

finito
